# Creation of Range FFT dataset

To handle the large size of data, this method saves each array as a seperate file and uses a key to store the labels and file name. Intended to be used later with a  Data Generator.
MTI filter applied.

In [0]:
APPLY_MTI_FILTER = False # whether or not to apply moving target indicator filter
WINDOW_LENGTH = 3 # 3 second window
STEP_SIZE = 100 # 0.1 second steps

In [2]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    

INTERIM_PATH = BASE_PATH + 'data/interim/'
PROCESSED_PATH = BASE_PATH + 'data/processed/range_FFT/'
if not os.path.exists(PROCESSED_PATH):
    os.makedirs(PROCESSED_PATH)

PROCESSED_PATH = PROCESSED_PATH + str(WINDOW_LENGTH)
if not os.path.exists(PROCESSED_PATH):
    os.makedirs(PROCESSED_PATH)
    
if APPLY_MTI_FILTER:
    PROCESSED_PATH = PROCESSED_PATH + '/MTI_applied/'
    
else:
    PROCESSED_PATH = PROCESSED_PATH + '/MTI_not_applied/'
    
if not os.path.exists(PROCESSED_PATH):
    os.makedirs(PROCESSED_PATH)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Running setup.py develop for src


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import mlab
from matplotlib import colors
from scipy.signal import butter, freqz, lfilter, spectrogram
import time
from sklearn import preprocessing

In [4]:
from keras.utils import np_utils
import sys
import csv
from keras.models import load_model
import pickle

Using TensorFlow backend.


### Function to aid processing Labels.csv

In [0]:
def find_label(movement):
    if movement == "Walking":
        return "walking"
    if movement == "Moving arm faster towards radar, slower away":
        return "pushing"
    if movement == "Sitting and standing":
        return "sitting"
    if movement == "Moving arm slower towards radar, faster away":
        return "pulling"
    if movement == "Circling arm forwards":
        return "circling"
    if movement == "Clapping":
        return "clapping"
    if movement == "Bending to pick up and back up":
        return "bending"


def identify_angle(angle):
    return angle.split()[0]


def is_on_place(angle):
    if len(angle.split()) > 2:
        return True
    return False


def assign_user_label(name):
    if name == "Aleksandar":
        return "A"
    if name == "Francesco":
        return "B"
    if name == "Nadezhda":
        return "C"
    if name == "Leila":
        return "D"
    if name == "Hadi":
        return "E"
    if name == "Ivelina":
        return "F"


def process_labels(df_labels):
    df_labels["label"] = df_labels.movement.apply(find_label)
    df_labels["user_label"] = df_labels.person.apply(assign_user_label)
    df_labels["aspect_angle"] = df_labels.angle.apply(identify_angle)
    df_labels["on_place"] = df_labels.angle.apply(is_on_place)
    return df_labels

### Functions for computing the Range FFT from the raw data

In [0]:
def nearest_odd_number(x):
    if(np.floor(x) % 2 == 0):
        return int(np.floor(x) + 1)
    else:
        return int(np.floor(x))

In [0]:
def MTI_filter(range_fft):
    # IIR Notch filter
    x = range_fft.shape[1]
    ns = nearest_odd_number(x) - 1
    data_range_MTI = np.zeros((range_fft.shape[0], ns), dtype=np.complex128)
    (b, a) = butter(4, 0.01, btype="high")
    # Apply Filter
    for i in range(range_fft.shape[0]):
        data_range_MTI[i, :ns] = lfilter(b, a, range_fft[i, :ns], axis=0)

    return data_range_MTI

In [0]:
def create_range_fft(radar_df, apply_MTI):
    # Grab RADAR settings from top of file
    center_frequency = float(radar_df.iloc[1]) # 5800000000Hz (5.6 GHz)
    sweep_time = float(radar_df.iloc[2])/1000  # convert to seconds (0.001 seconds)
    number_of_time_samples = float(radar_df.iloc[3]) # 128
    bandwidth = float(radar_df.iloc[4]) # 400000000Hz (400 MHz)
    sampling_frequency = number_of_time_samples/sweep_time
    '''
    record length = 60s
              = 60000 chirps with sweep time of 1ms
              = (7680000 measurments / 128 time samples) with sweep time of 1ms
    '''
    record_length = (len(radar_df.iloc[5:])/number_of_time_samples) * sweep_time

    number_of_chirps = record_length/sweep_time # 60000

    # Put data values into an array
    data = radar_df.iloc[5:].apply(complex).values

    # Reshape into chirps over time
    data_time = np.reshape(data, (int(number_of_chirps),int(number_of_time_samples)))    
    data_time = np.rot90(data_time, k=-1) # make axis 0 be chirp and axis 1 be time (makes visualization clearer)
    
    
    win = np.ones(data_time.shape)
    # Apply fast fourier transform should compute distance (range) from objects
    range_fft = np.fft.fft((data_time * win), axis=0)
    
    # take relevant half (other half appears to contain only noise)
    range_fft = range_fft[1:int(number_of_time_samples / 2), :]
    
    if apply_MTI:
        # Apply MTI filter to remove stationary objects
        range_fft = MTI_filter(range_fft)

    return np.rot90(range_fft) # undo earlier rotation to make time axis 0 and bin axis 1

### Generate dataset

In [0]:
labels = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
processed = {"A":{}, "B":{}, "C":{}, "D":{}, "E":{}, "F":{}}
for name, item in processed.items():
    for label in labels:
        item[label] = []

In [0]:
df_labels = pd.read_csv(INTERIM_PATH + 'Labels.csv')
df_labels.rename(columns={'dataset ID':'dataset_id'}, inplace=True)
df_labels = process_labels(df_labels)

In [0]:
# if program crashes halfway through, resume here
# LAST_ROW_COMPLETED = 0
LAST_ROW_COMPLETED = 113
SKIP = True

In [24]:
number_of_rows = df_labels.shape[0]
current_row = 1
for row in df_labels.itertuples():
    print(row.user_label, row.aspect_angle, row.label)
    if current_row > LAST_ROW_COMPLETED and SKIP:
        SKIP = False
        
    if row.aspect_angle != "0":
        current_row += 1
        continue

    print("Processing row", current_row, "of", number_of_rows)
    if not SKIP:
        file_name = INTERIM_PATH + "Dataset_" + str(row.dataset_id) + ".dat"

        radar_df = pd.read_csv(file_name, header=None)[1]

        # compute range fft from raw data
        range_fft = create_range_fft(radar_df, APPLY_MTI_FILTER)

        window_size = int(WINDOW_LENGTH * 1000)
        iterations = range_fft.shape[0] - window_size
        
    else:
        window_size = int(WINDOW_LENGTH * 1000)
        iterations = 60000 - window_size
        
    sub_label = 0
    file_names = []
    for i in range(0, iterations, STEP_SIZE):
        data_file_name = str(current_row) + "_" + str(sub_label) + ".npy"
        if not SKIP:
            np.save(PROCESSED_PATH + data_file_name, range_fft[i:(i + window_size),:])   
        file_names.append(data_file_name)
        sub_label += 1
        
    processed[row.user_label][row.label].append(file_names)

    current_row += 1

    
with open(PROCESSED_PATH + "index.pkl", "wb") as file:
    pickle.dump(processed, file)

A 0 walking
Processing row 1 of 123
A 0 walking
Processing row 2 of 123
A 30 walking
A 30 walking
A 60 walking
A 60 walking
A 0 pushing
Processing row 7 of 123
A 0 pushing
Processing row 8 of 123
A 30 pushing
A 30 pushing
A 60 pushing
A 60 pushing
A 0 sitting
Processing row 13 of 123
A 0 sitting
Processing row 14 of 123
A 0 pulling
Processing row 15 of 123
A 0 pulling
Processing row 16 of 123
A 0 circling
Processing row 17 of 123
A 0 circling
Processing row 18 of 123
A 0 clapping
Processing row 19 of 123
A 0 clapping
Processing row 20 of 123
A 0 bending
Processing row 21 of 123
A 0 bending
Processing row 22 of 123
B 0 walking
Processing row 23 of 123
B 0 walking
Processing row 24 of 123
B 0 pushing
Processing row 25 of 123
B 0 pushing
Processing row 26 of 123
B 0 sitting
Processing row 27 of 123
B 0 sitting
Processing row 28 of 123
B 0 circling
Processing row 29 of 123
B 0 circling
Processing row 30 of 123
B 0 clapping
Processing row 31 of 123
B 0 clapping
Processing row 32 of 123
B 0 